In [1]:
import os
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import joblib

# ============================
# 1. Config e caricamento dati
# ============================

CORPUS_JSON_PATH = "../data/processed/nba_players_corpus.json"
MODELS_DIR = "../data/models"

os.makedirs(MODELS_DIR, exist_ok=True)

# Carica il corpus
corpus_df = pd.read_json(CORPUS_JSON_PATH)
print("Corpus shape:", corpus_df.shape)
print(corpus_df.head(3))

# Controllo che ci sia text_profile
assert "text_profile" in corpus_df.columns, "Manca la colonna text_profile nel corpus!"

# Riempi eventuali NaN nel testo
corpus_df["text_profile"] = corpus_df["text_profile"].fillna("")

# ============================
# 2. Creazione vettore TF-IDF
# ============================

# Parametri base del vettorizzatore:
# - ngrammi fino a 2 parole per catturare espressioni tipo "tiratore da 3"
# - rimuovo parole troppo frequenti (max_df) e troppo rare (min_df)
vectorizer = TfidfVectorizer(
    lowercase=True,
    ngram_range=(1, 2),
    max_df=0.8,
    min_df=2,
    stop_words=None  
)


# Fit sul corpus
tfidf_matrix = vectorizer.fit_transform(corpus_df["text_profile"])

print("TF-IDF matrix shape:", tfidf_matrix.shape)

# ============================
# 3. Salvataggio dell'indice
# ============================

VECTORIZER_PATH = os.path.join(MODELS_DIR, "tfidf_vectorizer.joblib")
MATRIX_PATH = os.path.join(MODELS_DIR, "tfidf_matrix.joblib")
META_PATH = os.path.join(MODELS_DIR, "index_metadata.csv")

joblib.dump(vectorizer, VECTORIZER_PATH)
joblib.dump(tfidf_matrix, MATRIX_PATH)

# Metadati allineati alle righe della matrice TF-IDF
meta_cols = []
for col in ["player_id", "Player", "DraftYear", "Pick", "PickBand", "Pos", "text_profile"]:
    if col in corpus_df.columns:
        meta_cols.append(col)

metadata_df = corpus_df[meta_cols].reset_index(drop=True)
metadata_df.to_csv(META_PATH, index=False)

print("Indice salvato in:")
print(" -", VECTORIZER_PATH)
print(" -", MATRIX_PATH)
print(" -", META_PATH)

# ============================
# 4. Funzione di ricerca di test
# ============================

def search_players(query, top_k=10):
    """
    Esegue una prima ricerca semplice:
    - trasforma la query in TF-IDF
    - calcola similarità coseno con tutti i giocatori
    - restituisce i top_k più simili
    """
    if not query or not isinstance(query, str):
        raise ValueError("La query deve essere una stringa non vuota.")

    # Trasforma la query nello stesso spazio TF-IDF
    query_vec = vectorizer.transform([query])

    # Similarità coseno con tutti i documenti
    sim_scores = cosine_similarity(query_vec, tfidf_matrix).ravel()

    # Ordina dal più simile al meno simile
    top_idx = np.argsort(sim_scores)[::-1][:top_k]

    results = metadata_df.iloc[top_idx].copy()
    results["similarity"] = sim_scores[top_idx]

    return results

# ============================
# 5. Test rapido
# ============================

test_query = "cerco una guardia tiratrice con ottime percentuali da 3 e buoni tiri liberi, forte rimbalzista offensivo"
results = search_players(test_query, top_k=5)

pd.set_option("display.max_colwidth", 200)
print("Query di test:", test_query)

# Mostra solo le colonne che ESISTONO davvero
cols_to_show = [c for c in ["Player", "DraftYear", "Pick", "Pos", "similarity", "text_profile"] if c in results.columns]
display(results[cols_to_show])



Corpus shape: (8323, 7)
            Player  DraftYear  Pick PickBand  \
0  Clifton McNeely       1947     1    Top10   
1       Glen Selbo       1947     2    Top10   
2     Bulbs Ehlers       1947     3    Top10   

                                        text_profile   Status  player_id  
0  Giocatore: Clifton McNeely. Provenienza: Texas...  Retired          0  
1  Giocatore: Glen Selbo. Provenienza: Wisconsin....  Retired          1  
2  Giocatore: Bulbs Ehlers. Provenienza: Purdue. ...  Retired          2  
TF-IDF matrix shape: (8323, 11236)
Indice salvato in:
 - ../data/models/tfidf_vectorizer.joblib
 - ../data/models/tfidf_matrix.joblib
 - ../data/models/index_metadata.csv
Query di test: cerco una guardia tiratrice con ottime percentuali da 3 e buoni tiri liberi, forte rimbalzista offensivo


,Player,DraftYear,Pick,similarity,text_profile
1752,Gail Goodrich,1965,3,0.304001,"Giocatore: Gail Goodrich. Provenienza: UCLA. Scelto al Draft NBA 1965 con la pick 3. Fascia di scelta: Top10. Tiro da 3: -100.0%, tiratore da 3 punti poco affidabile. Tiri liberi: 80.7%, buon tira..."
714,Bob Pettit,1954,5,0.298832,"Giocatore: Bob Pettit. Provenienza: LSU. Scelto al Draft NBA 1954 con la pick 5. Fascia di scelta: Top10. Tiro da 3: -100.0%, tiratore da 3 punti poco affidabile. Tiri liberi: 76.1%, buon tiratore..."
1471,John Havlicek,1962,9,0.293934,"Giocatore: John Havlicek. Provenienza: Ohio State. Scelto al Draft NBA 1962 con la pick 9. Fascia di scelta: Top10. Tiro da 3: -100.0%, tiratore da 3 punti poco affidabile. Tiri liberi: 81.5%, buo..."
2569,Rudy Tomjanovich,1970,2,0.285437,"Giocatore: Rudy Tomjanovich. Provenienza: Michigan. Scelto al Draft NBA 1970 con la pick 2. Fascia di scelta: Top10. Tiro da 3: -100.0%, tiratore da 3 punti poco affidabile. Tiri liberi: 78.4%, bu..."
1863,Dave Bing,1966,2,0.284868,"Giocatore: Dave Bing. Provenienza: Syracuse. Scelto al Draft NBA 1966 con la pick 2. Fascia di scelta: Top10. Tiro da 3: -100.0%, tiratore da 3 punti poco affidabile. Tiri liberi: 77.5%, buon tira..."


Cosa hai ora con lo Step B

Un indice TF-IDF persistente (tfidf_vectorizer.joblib + tfidf_matrix.joblib).

Un file index_metadata.csv con tutti i dati allineati a ogni riga della matrice.

Una prima funzione search_players() che, già così, ti permette di fare:

search_players("cerco centro dominante a rimbalzo e buon difensore", top_k=10)